In [ ]:
import pennylane as qml
from qiskit_ibm_runtime.fake_provider import FakeQuebec
from pennylane_qiskit import qiskit_session
from qiskit_ibm_runtime import QiskitRuntimeService


import joblib
import click
import json
import time
import os
import itertools
import collections.abc
from shutil import copy,SameFileError
from glob import glob

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import pennylane as qml
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
# from qiskit_ibm_provider import IBMProvider

from quantum.Quantum import QuantumRegressor
from quantum.Evaluate import evaluate
from settings import ANSATZ_LIST, ENCODER_LIST
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import InconsistentVersionWarning

In [ ]:
ANSATZs=[]
ENCODERs=[]
pairs=[]


for i in glob(os.path.join(os.path.expanduser('~'),'qregress/function-calc-test/RUD_AL/3_AL_lin16qubit/*/*results.json')):
    print(i)
    pair = os.path.basename(i).replace('_results.json','')
    pairs.append(pair)
    an, en = pair.split('_')
    ANSATZs.append(an)
    ENCODERs.append(en)
    


ANSATZs=set(ANSATZs)
ENCODERs=set(ENCODERs)

In [ ]:
# device='qulacs.simulator'
# num_qubits=16
# dev = qml.device(device, wires=num_qubits)

# service = QiskitRuntimeService(channel="ibm_quantum", instance='pinq-quebec-hub/univ-toronto/default')
# _backend = service.least_busy(operational=True, simulator=False, min_num_qubits=127)
_backend = FakeQuebec()

dev = qml.device('qiskit.remote', wires=127, backend=_backend, resilience_level=1, optimization_level=1, seed_transpiler=42)
# target = _backend.target

num_qubits = 16

In [ ]:
dev.backend.backend_name

In [ ]:
@qml.qnode(dev)
def circuit(features, parameters):
    #  builds the circuit with the given encoder and variational circuits.
    #  encoder and variational circuits must have only two required parameters, params/feats and wires
    for i in range(re_upload_depth):
        params = initial_parameters.reshape(re_upload_depth,-1)[i]
        encoder(features, wires=range(num_qubits))
        variational(params, wires=range(num_qubits))

    return qml.expval(qml.PauliZ(0))

In [ ]:
ENCODER, ANSATZ = "IQP","Full-Pauli-CRZ"
LAYERS = 1 
re_upload_depth = 1

In [ ]:
encoder = ENCODER_LIST[ENCODER]
variational=ANSATZ_LIST[ANSATZ]
variational.layers = LAYERS
variational.set_wires(range(num_qubits))


num_params = variational.num_params * re_upload_depth
generator = np.random.default_rng(12958234)
initial_parameters = generator.uniform(-np.pi, np.pi, num_params)
# feat = generator.uniform(-1, 1, num_qubits)

In [ ]:
with open('linear_train.bin','rb') as f:
    train = joblib.load(f)

with open('linear_test.bin','rb') as f:
    test = joblib.load(f)

with open('linear_scaler.bin','rb') as f:
    scaler = joblib.load(f)
X_train, y_train = train['X'],train['y']
X_test, y_test = test['X'],test['y']


In [ ]:
def predict(x,params):
    return np.array()

In [ ]:
output = []

with qiskit_session(dev) as session:
    for i in range(0,2):
        for x in X_train:
            output.append(circuit(x,initial_parameters))

In [ ]:
output